# Import modules

In [1]:
#From our previous codes
%matplotlib inline
import xarray as xr
import glob
import os
import matplotlib.pyplot as plt
import pandas.plotting
import numpy as np
from mpl_toolkits.basemap import Basemap
import netCDF4
from netCDF4 import Dataset
import pandas as pd
import re

#From Katie's code
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam
from keras.regularizers import l2

import keras.backend as K

from scipy import stats

Using TensorFlow backend.


# Read the simulations.csv file

In [76]:
# Read data from file 'simulations.csv' 
# Then arrange ncely the data in dataframe and change data type of the values
data = pd.read_csv("./simulations.csv")  

line0 = data.iloc[0].to_string()
param_names = re.findall( r'( .*?)=', line0)

df = pd.DataFrame(param_values, index = param_names)

i=0
while i<199:
    line = data.iloc[i].to_string()
    param_values = re.findall( r'=(\d.*?e.*?0\d)', line)
    df[str(i)]=param_values
    i = i+1
    
df

0         0         1         2         3         4  \
   RKDX         4.96e-05  4.96e-05  4.96e-05  4.96e-05  4.96e-05  4.96e-05   
   TENTR        5.87e-04  5.87e-04  5.87e-04  5.87e-04  5.87e-04  5.87e-04   
   TENTRX       1.02e-05  1.02e-05  1.02e-05  1.02e-05  1.02e-05  1.02e-05   
   VVX          5.82e-05  5.82e-05  5.82e-05  5.82e-05  5.82e-05  5.82e-05   
   RAUTEFR      2.06e-03  2.06e-03  2.06e-03  2.06e-03  2.06e-03  2.06e-03   
   RAUTEFS      3.50e-03  3.50e-03  3.50e-03  3.50e-03  3.50e-03  3.50e-03   
   RQICRMIN     1.10e-07  1.10e-07  1.10e-07  1.10e-07  1.10e-07  1.10e-07   
   RQICRMAX     1.06e-05  1.06e-05  1.06e-05  1.06e-05  1.06e-05  1.06e-05   
   RQLCR        1.47e-04  1.47e-04  1.47e-04  1.47e-04  1.47e-04  1.47e-04   
   TFVI         1.04e-01  1.04e-01  1.04e-01  1.04e-01  1.04e-01  1.04e-01   
   TFVL         7.01e-02  7.01e-02  7.01e-02  7.01e-02  7.01e-02  7.01e-02   
   RACCEF       1.13e+00  1.13e+00  1.13e+00  1.13e+00  1.13e+00  1.13e+00   
   RRIMEF       9.91e-01  9.91e-01  9.91e-01  9.91e-01  9.91e-01  9.91e-01   
   RAGGEF       2.88e-01  2.88e-01  2.88e-01  2.88e-01  2.88e-01  2.88e-01   
   REVASX       1.05e-07  1.05e-07  1.05e-07  1.05e-07  1.05e-07  1.05e-07   
   RREVASXCS    5.95e-01  5.95e-01  5.95e-01  5.95e-01  5.95e-01  5.95e-01   
   RSWINHF_LIQ  6.46e-01  6.46e-01  6.46e-01  6.46e-01  6.46e-01  6.46e-01   
   RSWINHF_ICE  8.64e-01  8.64e-01  8.64e-01  8.64e-01  8.64e-01  8.64e-01   
   RLWINHF_LIQ  5.84e-01  5.84e-01  5.84e-01  5.84e-01  5.84e-01  5.84e-01   
   RLWINHF_ICE  6.03e-01  6.03e-01  6.03e-01  6.03e-01  6.03e-01  6.03e-01   
       RKDN     4.04e-05  2.00e-05  2.84e-05  3.79e-05  3.60e-05  2.42e-05   
   RKDX         3.29e-04  1.80e-04  1.38e-04  1.63e-04  7.87e-04  3.11e-04   
   TENTR        1.35e-05  1.86e-05  1.79e-05  1.66e-05  4.53e-06  1.41e-05   

                       5         6         7         8  ...       189  \
   RKDX         4.96e-05  4.96e-05  4.96e-05  4.96e-05  ...  4.96e-05   
   TENTR        5.87e-04  5.87e-04  5.87e-04  5.87e-04  ...  5.87e-04   
   TENTRX       1.02e-05  1.02e-05  1.02e-05  1.02e-05  ...  1.02e-05   
   VVX          5.82e-05  5.82e-05  5.82e-05  5.82e-05  ...  5.82e-05   
   RAUTEFR      2.06e-03  2.06e-03  2.06e-03  2.06e-03  ...  2.06e-03   
   RAUTEFS      3.50e-03  3.50e-03  3.50e-03  3.50e-03  ...  3.50e-03   
   RQICRMIN     1.10e-07  1.10e-07  1.10e-07  1.10e-07  ...  1.10e-07   
   RQICRMAX     1.06e-05  1.06e-05  1.06e-05  1.06e-05  ...  1.06e-05   
   RQLCR        1.47e-04  1.47e-04  1.47e-04  1.47e-04  ...  1.47e-04   
   TFVI         1.04e-01  1.04e-01  1.04e-01  1.04e-01  ...  1.04e-01   
   TFVL         7.01e-02  7.01e-02  7.01e-02  7.01e-02  ...  7.01e-02   
   RACCEF       1.13e+00  1.13e+00  1.13e+00  1.13e+00  ...  1.13e+00   
   RRIMEF       9.91e-01  9.91e-01  9.91e-01  9.91e-01  ...  9.91e-01   
   RAGGEF       2.88e-01  2.88e-01  2.88e-01  2.88e-01  ...  2.88e-01   
   REVASX       1.05e-07  1.05e-07  1.05e-07  1.05e-07  ...  1.05e-07   
   RREVASXCS    5.95e-01  5.95e-01  5.95e-01  5.95e-01  ...  5.95e-01   
   RSWINHF_LIQ  6.46e-01  6.46e-01  6.46e-01  6.46e-01  ...  6.46e-01   
   RSWINHF_ICE  8.64e-01  8.64e-01  8.64e-01  8.64e-01  ...  8.64e-01   
   RLWINHF_LIQ  5.84e-01  5.84e-01  5.84e-01  5.84e-01  ...  5.84e-01   
   RLWINHF_ICE  6.03e-01  6.03e-01  6.03e-01  6.03e-01  ...  6.03e-01   
       RKDN     3.74e-05  2.53e-05  3.62e-05  4.72e-05  ...  3.88e-05   
   RKDX         4.67e-04  8.36e-04  6.21e-04  7.30e-04  ...  6.62e-04   
   TENTR        1.58e-05  4.28e-06  1.32e-05  1.72e-05  ...  7.69e-06   

                     190       191       192       193       194       195  \
   RKDX         4.96e-05  4.96e-05  4.96e-05  4.96e-05  4.96e-05  4.96e-05   
   TENTR        5.87e-04  5.87e-04  5.87e-04  5.87e-04  5.87e-04  5.87e-04   
   TENTRX       1.02e-05  1.02e-05  1.02e-05  1.02e-05  1.02e-05  1.02e-05   
   VVX          5.82e-05  5.82e-05  5.82e-05  5.82e-05  5.82e-05  5.82e-05   
   RAUT

In [71]:
data.head()

RKDN=4.96e-05   RKDX=5.87e-04   TENTR=1.02e-05   TENTRX=5.82e-05   VVX=-3.25e+01   RAUTEFR=2.06e-03   RAUTEFS=3.50e-03   RQICRMIN=1.10e-07   RQICRMAX=1.06e-05   RQLCR=1.47e-04   TFVI=1.04e-01   TFVL=7.01e-02   RACCEF=1.13e+00   RRIMEF=9.91e-01   RAGGEF=2.88e-01   REVASX=1.05e-07   RREVASXCS=5.95e-01   RSWINHF_LIQ=6.46e-01   RSWINHF_ICE=8.64e-01   RLWINHF_LIQ=5.84e-01   RLWINHF_ICE=6.03e-01   
0  RKDN=2.00e-05   RKDX=1.80e-04   TENTR=1.86e-05...                                                                                                                                                                                                                                                                                                                                                         
1  RKDN=2.84e-05   RKDX=1.38e-04   TENTR=1.79e-05...                                                                                                                                                                                                                                                                                                                                                         
2  RKDN=3.79e-05   RKDX=1.63e-04   TENTR=1.66e-05...                                                                                                                                                                                                                                                                                                                                                         
3  RKDN=3.60e-05   RKDX=7.87e-04   TENTR=4.53e-06...                                                                                                                                                                                                                                                                                                                                                         
4  RKDN=2.42e-05   RKDX=3.11e-04   TENTR=1.41e-05...